---
# <center>**_Excel Sheet Automation_**
---

##### <u>*__Notes!__*</u>

---

In [ ]:
import os

current_dir = os.getcwd()
print(os.listdir(current_dir))


### Note -
We have a excel sheet & we wanna copy the data from one excel sheet & paste it in another. <br>
The Copying & pasting can be done in 2 ways - 
```
a. Copy - from particular cells in the excel sheet.
b. Paste - to particular cells in the excel sheet.


---

# Step-by-Step Explanation of `build_folder_dict` Function

This document provides a detailed breakdown of the `build_folder_dict` function, which creates a dictionary representing the structure of a folder, including its files and subfolders.

## 1. Importing `pathlib`

```python
from pathlib import Path
```

- `pathlib` is a Python module for handling file paths in a modern, object-oriented way.
- `Path` is the main class used to represent file or folder paths.
- It’s cross-platform (works on Windows, macOS, Linux) and is more convenient than older modules like `os.path`.

## 2. Defining the Function

```python
def build_folder_dict(folder_path):
```

- The function `build_folder_dict` takes a `folder_path` (a string, e.g., `"my_folder"`) as input.
- Its purpose is to return a dictionary mapping folder names to their contents (files and subfolders).

## 3. Initializing the Dictionary

```python
folder_dict = {}
```

- `folder_dict` is the main dictionary that will store the folder structure.
- Each key will be a folder’s relative path (as a string), and each value will be a dictionary with two keys:
  - `'files'`: A list of file names in that folder.
  - `'subfolders'`: A nested dictionary for subfolders.

## 4. Converting the Input Path

```python
root_path = Path(folder_path)
```

- `Path(folder_path)` converts the input string (e.g., `"my_folder"`) into a `Path` object.
- `root_path` represents the top-level folder to be scanned. It’s used to compute relative paths for all items.

## 5. Recursive Iteration with `rglob`

```python
try:
    for item in root_path.rglob('*'):
```

- `root_path.rglob('*')` is a `pathlib` method that recursively iterates through all files and subfolders in `root_path`.
- The `'*'` pattern matches all items (files and folders) at any depth.
- Each `item` is a `Path` object representing a file or folder (e.g., `my_folder/file1.txt` or `my_folder/subfolder1`).
- The `try` block catches errors like invalid folder paths or permission issues.

## 6. Getting the Parent Folder

```python
parent = item.parent.relative_to(root_path) if item.parent != root_path else Path('.')
parent_name = str(parent)
```

- `item.parent` is the folder containing `item` (e.g., for `my_folder/subfolder1/file3.txt`, it’s `my_folder/subfolder1`).
- `relative_to(root_path)` computes the parent’s path relative to the root (e.g., `subfolder1` for `my_folder/subfolder1`).
- If `item.parent == root_path` (i.e., the item is directly in the root folder), we use `Path('.')` to represent the root folder as `"."`.
- `parent_name = str(parent)` converts the path to a string (e.g., `"subfolder1"` or `"."`) for use as a dictionary key.

## 7. Initializing Folder Entry

```python
if parent_name not in folder_dict:
    folder_dict[parent_name] = {'files': [], 'subfolders': {}}
```

- If the parent folder (e.g., `"subfolder1"`) isn’t in `folder_dict`, we create an entry.
- The entry is a dictionary with:
  - `'files'`: An empty list to store file names.
  - `'subfolders'`: An empty dictionary to store subfolder information.

## 8. Handling Files

```python
if item.is_file():
    try:
        with item.open('r', encoding='utf-8') as f:
            pass  # We don't need to read content, just verify access
        folder_dict[parent_name]['files'].append(item.name)
    except (UnicodeDecodeError, PermissionError, IOError):
        continue  # Ignore files that can't be read
```

- `item.is_file()` checks if the current `item` is a file (not a folder).
- We attempt to open the file in text mode (`'r'`) with `utf-8` encoding to verify it’s readable.
- `item.open('r', encoding='utf-8')` tries to open the file.
- `pass` means we don’t read the content; we just ensure it opens without errors.
- If successful, we add the file’s name (`item.name`, e.g., `"file1.txt"`) to the `'files'` list of the parent folder.
- If an error occurs (e.g., `UnicodeDecodeError` for binary files, `PermissionError` for locked files), we skip the file with `continue`.

## 9. Handling Subfolders

```python
elif item.is_dir():
    subfolder = item.relative_to(root_path)
    subfolder_name = str(subfolder)
    if subfolder_name not in folder_dict:
        folder_dict[subfolder_name] = {'files': [], 'subfolders': {}}
```

- `item.is_dir()` checks if the current `item` is a folder.
- `item.relative_to(root_path)` gets the folder’s path relative to the root (e.g., `"subfolder1/subfolder2"`).
- `subfolder_name = str(subfolder)` converts it to a string for the dictionary key.
- If the subfolder isn’t in `folder_dict`, we create an entry with empty `'files'` and `'subfolders'` fields.
- This ensures every folder (even empty ones) is represented in the dictionary.

## 10. Error Handling for the Folder

```python
except Exception as e:
    print(f"Error accessing folder {folder_path}: {e}")
```

- If there’s a problem accessing the folder (e.g., it doesn’t exist or lacks permissions), we print an error and continue.
- The function will still return `folder_dict` (possibly empty if no items were processed).

## 11. Returning the Result

```python
return folder_dict
```

- The function returns `folder_dict`, which contains the full folder structure.

## 12. Example Usage and Pretty Printing

```python
folder_path = "path/to/your/folder"
result = build_folder_dict(folder_path)
import json
print(json.dumps(result, indent=2))
```

- `folder_path` is the path to the folder you want to scan (replace with your actual path).
- `build_folder_dict(folder_path)` runs the function and returns the dictionary.
- `json.dumps(result, indent=2)` prints the dictionary in a readable, indented format.

## Example Folder Structure and Output

Suppose your folder is:

```
my_folder/
├── file1.txt
├── file2.bin (unreadable)
├── subfolder1/
│   ├── file3.txt
│   ├── subfolder2/
│   │   ├── file4.txt
```

Running the code produces a dictionary like:

```json
{
  ".": {
    "files": ["file1.txt"],
    "subfolders": {
      "subfolder1": {
        "files": ["file3.txt"],
        "subfolders": {
          "subfolder1/subfolder2": {
            "files": ["file4.txt"],
            "subfolders": {}
          }
        }
      }
    }
  },
  "subfolder1": {
    "files": ["file3.txt"],
    "subfolders": {
      "subfolder1/subfolder2": {
        "files": ["file4.txt"],
        "subfolders": {}
      }
    }
  },
  "subfolder1/subfolder2": {
    "files": ["file4.txt"],
    "subfolders": {}
  }
}
```

## Key Points

- **Why `pathlib`?** It simplifies path handling (no `os.path.join` needed) and is more readable. `rglob` makes recursion easy.
- **Dictionary Structure:** Each folder is a key, with a value containing `'files'` (list of readable files) and `'subfolders'` (nested dictionary).
- **Skipping Unreadable Files:** The `try-except` block ensures binary files, locked files, or other inaccessible files are ignored.
- **Relative Paths:** Using `relative_to` keeps folder names clean (e.g., `"subfolder1"` instead of full paths).
- **Empty Folders:** They’re included with empty `'files'` and `'subfolders'` fields.

## Potential Tweaks

- **Filter Extensions:** Add `if item.suffix == '.txt'` in the `is_file()` block to only include specific file types.
- **Include Binary Files:** Change `open('r', encoding='utf-8')` to `open('rb')` to include all files.
- **Simplify Structure:** If you don’t need the `'subfolders'` nesting, the code can be modified to flatten it.

😊

In [ ]:
from pathlib import Path
from rich import print as print_

def build_folder_dict(folder_path):
    # Initialize the main dictionary
    folder_dict = {}
    
    # Convert input path to Path object
    root_path = Path(folder_path)
    
    try:
        # Iterate through all items in the folder recursively
        for item in root_path.rglob('*'):
            # Get the parent folder name (relative to root_path)
            parent = item.parent.relative_to(root_path) if item.parent != root_path else Path('.')
            parent_name = str(parent)
            
            # Initialize dictionary entry for parent folder if not exists
            if parent_name not in folder_dict:
                folder_dict[parent_name] = {'files': [], 'subfolders': {}}
            
            # If item is a file, add to files list
            if item.is_file():
                try:
                    # Attempt to access file to ensure it's readable
                    with item.open('r', encoding='utf-8') as f:
                        pass  # We don't need to read content, just verify access
                    folder_dict[parent_name]['files'].append(item.name)
                except (UnicodeDecodeError, PermissionError, IOError):
                    continue  # Ignore files that can't be read
            
            # If item is a directory, ensure it's represented in subfolders
            elif item.is_dir():
                # Relative path of the subfolder
                subfolder = item.relative_to(root_path)
                subfolder_name = str(subfolder)
                if subfolder_name not in folder_dict:
                    folder_dict[subfolder_name] = {'files': [], 'subfolders': {}}
    
    except Exception as e:
        print(f"Error accessing folder {folder_path}: {e}")
    
    return folder_dict

# Example usage
folder_path = "D:/Docs/01_Projects/27_SH_Huawei_MKC2_MLC55"  # Replace with your folder path
result = build_folder_dict(folder_path)

# Print the dictionary in a readable format
import json
print(json.dumps(result, indent=2))
print_(result)


---

### Folder structure visualizer

In [ ]:
import json
from pathlib import Path
from typing import Dict, Union
from rich import print as print_

# Why we use the typing module:
# The typing module helps us tell Python what kinds of data (like strings, lists, or dictionaries) a function expects
# or returns. This makes our code clearer, easier to understand, and helps catch mistakes early, especially when
# working with complex data like dictionaries that hold lists or other dictionaries.

# About Dict and Union:
# - Dict[str, Union[list, dict]]: This means a dictionary where keys are strings, and values can be either a list
#   (like a list of file names) or another dictionary (for subfolders). Union[list, dict] allows the value to be one
#   of these types, making our code flexible but clear.

def gen(text: str, style: str) -> str:
    """
    Generates a styled string for printing with rich.

    This function wraps text in a style tag for colorful output using the rich library. For example, you can make
    text bold and red or blue and italic. It's like adding decorations to text to make it stand out.

    Args:
        text (str): The text you want to style.
        style (str): The style to apply (e.g., 'bold red', 'blue', 'cyan').

    Returns:
        str: The text wrapped in rich style tags, like '[bold red]text[/bold red]'.

    Example:
        >>> print_(gen("Hello!", "bold blue"))
        [bold blue]Hello![/bold blue]
    """
    return f"[{style}]{text}[/{style}]"

def build_folder_dict(folder_path: str) -> Dict[str, Dict[str, Union[list, dict]]]:
    """
    Builds a dictionary showing the structure of a folder and its contents.

    This function looks through a folder and all its subfolders to create a dictionary. Each key in the dictionary
    is a folder path (relative to the starting folder), and its value is another dictionary that lists files and
    subfolders. It only includes files we can read and skips anything that causes errors.

    Args:
        folder_path (str): The path to the folder you want to scan (e.g., "C:/MyFolder").

    Returns:
        Dict[str, Dict[str, Union[list, dict]]]: A dictionary where:
            - Keys are folder paths (as strings).
            - Each value is a dictionary with:
                - 'files': A list of file names in that folder.
                - 'subfolders': A dictionary of subfolders (same structure as the main dictionary).

    Example:
        >>> result = build_folder_dict("D:/Docs/")
        >>> print(json.dumps(result, indent=2))
    """
    # Create an empty dictionary to store the folder structure
    folder_dict: Dict[str, Dict[str, Union[list, dict]]] = {}

    # Turn the folder path into a Path object (makes it easier to work with files and folders)
    try:
        root_path = Path(folder_path).resolve()  # Resolve ensures we get the full, correct path
    except (PermissionError, FileNotFoundError, OSError) as e:
        # If we can't access the folder, print an error in bold red and return an empty dictionary
        print_(gen(f"Error: Can't access folder {folder_path}: {e}", "bold red"))
        return folder_dict

    # Check if the path is actually a folder
    if not root_path.is_dir():
        print_(gen(f"Error: {folder_path} is not a folder", "bold red"))
        return folder_dict

    # Keep track of folders we've already added to avoid repeating work
    processed_dirs = set()

    # Look through all files and folders inside the root folder (and its subfolders)
    for item in root_path.rglob('*'):
        # Skip items we can't access
        try:
            if not item.exists():
                continue
        except (PermissionError, OSError):
            continue

        # Get the parent folder's path (relative to the root folder)
        parent = item.parent.relative_to(root_path) if item.parent != root_path else Path('.')
        parent_name = str(parent)  # Turn the path into a string

        # Add the parent folder to our dictionary if we haven't yet
        if parent_name not in processed_dirs:
            folder_dict[parent_name] = {'files': [], 'subfolders': {}}
            processed_dirs.add(parent_name)

        # If the item is a file, try to add it to the files list
        if item.is_file():
            try:
                # Check if we can read the file (we only read a tiny bit to be fast)
                with item.open('rb') as f:
                    f.read(1024)  # Read 1KB to make sure the file is accessible
                folder_dict[parent_name]['files'].append(item.name)
            except (UnicodeDecodeError, PermissionError, IOError):
                # Skip files we can't read (e.g., binary files or locked files)
                continue

        # If the item is a folder, add it to the dictionary
        elif item.is_dir():
            subfolder = item.relative_to(root_path)  # Get the folder's path relative to root
            subfolder_name = str(subfolder)
            if subfolder_name not in processed_dirs:
                folder_dict[subfolder_name] = {'files': [], 'subfolders': {}}
                processed_dirs.add(subfolder_name)

    return folder_dict

def print_ascii_tree_from_dict(folder_dict: Dict[str, Dict[str, Union[list, dict]]], prefix: str = "") -> None:
    """
    Prints an ASCII tree of the folder structure from a dictionary.

    This function takes a folder dictionary (like the one from build_folder_dict) and prints it as a
    tree using ASCII characters. Folders are shown in bold blue, files in green, and subfolders in cyan
    to make it easy to see the structure.

    Args:
        folder_dict (Dict[str, Dict[str, Union[list, dict]]]): The dictionary with folder structure.
        prefix (str): The string of ASCII characters used to indent the tree (used when we go deeper).

    Returns:
        None: Just prints the tree to the console.

    Example:
        >>> folder_dict = build_folder_dict("D:/Docs/")
        >>> print_ascii_tree_from_dict(folder_dict)
    """
    # Sort the folders so the output is always in the same order
    for folder, content in sorted(folder_dict.items()):
        # Print the folder name in bold blue with a └── symbol
        print_(gen(f"{prefix}└── {folder}/", "bold blue"))
        
        # Print all files in this folder in green with a ├── symbol
        for file in sorted(content['files']):
            print_(gen(f"{prefix}    ├── {file}", "green"))
        
        # Find subfolders that are one level deeper than the current folder
        subfolders = {k: v for k, v in folder_dict.items() if k.startswith(f"{folder}/") and k != folder}
        for subfolder, subcontent in sorted(subfolders.items()):
            # Only print subfolders one level deep to avoid repeating parts of the tree
            if subfolder.count('/') == folder.count('/') + 1:
                # Print subfolder name in cyan
                print_(gen(f"{prefix}    └── {subfolder.split('/')[-1]}/", "cyan"))
                # Print files in this subfolder in green
                for file in sorted(subcontent['files']):
                    print_(gen(f"{prefix}        ├── {file}", "green"))

def print_ascii_tree_from_json(json_path: str) -> None:
    """
    Prints an ASCII tree of the folder structure from a JSON file.

    This function reads a JSON file that contains a folder dictionary (like the one from build_folder_dict)
    and prints it as an ASCII tree. Folders are in bold blue, files in green, and subfolders in cyan.

    Args:
        json_path (str): Path to the JSON file with the folder structure.

    Returns:
        None: Just prints the tree to the console.

    Example:
        >>> print_ascii_tree_from_json("folder_structure.json")
    """
    # Try to read the JSON file
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            folder_dict = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError, PermissionError) as e:
        # If we can't read the JSON file, print an error in bold red
        print_(gen(f"Error: Can't read JSON file {json_path}: {e}", "bold red"))
        return

    # Use the dictionary-based function to print the tree
    print_ascii_tree_from_dict(folder_dict)

if __name__ == "__main__":
    # Example usage: scan a folder and print its structure
    folder_path = "D:/Docs/01_Projects/"  # Change this to your folder path
    result = build_folder_dict(folder_path)

    # Print the dictionary as JSON for debugging
    print_(gen("\nJSON Output:", "bold"))
    print(json.dumps(result, indent=2))

    # Print the dictionary as an ASCII tree
    print_(gen("\nFolder Structure as ASCII Tree:", "bold"))
    print_ascii_tree_from_dict(result)

    # Save the dictionary to a JSON file and print it as a tree
    json_path = "folder_structure.json"
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(result, f, indent=2)
    print_(gen("\nFolder Structure from JSON File:", "bold"))
    print_ascii_tree_from_json(json_path)